In [1]:
from selenium import webdriver
import sys,os,os.path
import json
import time
import random
import json

In [19]:
#sys.path.append(os.path.expanduser('/home/nad/geckodriver'))
os.environ['PATH']+=':'+'/home/nad/geckodriver'
browser = webdriver.Firefox()
browser.implicitly_wait(10)

In [20]:
browser.get('https://www.facebook.com/nadhemm')

In [26]:
def extract_personal_info(name):
    info = {}
    positions = []
    prev_schools = []
    for el in browser.find_elements_by_class_name("textContent"):
        text = el.text
        split = text.split(" at ")
        if len(split)>1:
            if "Studies" in text:                
                info['school'] = split[1]
                if "Studies at" not in text: # we can extract the field
                    info['field'] = split[0][len("Studies "):]
            elif "Studied" in text:
                prev_studies = {}
                prev_studies['prev_school'] = split[1]
                if "Studied at" not in text: # we can extract the field
                    prev_studies['prev_field'] = split[0][len("Studied "):]
                prev_schools.append(prev_studies)
            else:
                positions.append({'position': split[0] ,
                                 'organization': split[1]})
        elif "Went to " in text:
            prev_schools.append({'prev_school':text[len("Went to "):]})
        elif "Lives in " in text:
            info['address'] = text[len("Lives in "):]
        elif "From " in text:
            info['birth_address'] = text[len("From "):]        
        elif "Followed by " in text:
            try:
                info['nb_followers'] = text.split()[2]        
            except Exception as e:
                print("Cannot extract followers, details:", e)
    info['positions'] = positions
    info['prev_schools'] = prev_schools
    return info

In [22]:
def extract_name():
    return browser.find_element_by_id("fb-timeline-cover-name").text

In [23]:
with open('friends_links.json', 'r') as f:
    links = json.load(f)['friends']

In [ ]:
profiles = []
start,end = 121,140
cnt = start
for link in links[start:end+1]:
    print("Processing profile", cnt, "out of", end)
    profile = {}
    try:
        browser.get(link)
        try:
            name = extract_name()
            profile['name'] = name
            try:
                profile.update(extract_personal_info(name))
            except Exception as e:
                print("Error occurred while extracting info from user %s, details:"%name, e)
        except Exception as e:
            print("Error occurred while extracting name, link:\n %s\n  details:"%link, e)
    except Exception as e:
        print("Invalid link", link)
    profiles.append(profile)
    cnt+=1
    if cnt %20 ==0:
        with open('friends_data/friends_data_page'+str(int(cnt/20))+'.json', 'w') as f:
            json.dump(profiles, f, ensure_ascii=False)
        profiles =[]
    time.sleep(1+random.random())
    if cnt%10 == 0:
        time.sleep(5)
        

Processing profile 121 out of 140
Processing profile 122 out of 140
Processing profile 123 out of 140
Processing profile 124 out of 140
Processing profile 125 out of 140
Processing profile 126 out of 140
Processing profile 127 out of 140
Processing profile 128 out of 140
Processing profile 129 out of 140


In [30]:
links[61]

'https://www.facebook.com/Ahmed.Amine15?fref=pb&hc_location=friends_tab'

In [16]:
with open('friends_data.json', 'w') as f:
    json.dump(profiles, f, ensure_ascii=False)

In [102]:
import pandas
df = pandas.read_json('friends_data.json')


In [103]:
df

,address,birth_address,name,positions,prev_schools
0,"Munich, Germany",Sfax,Nassim Walha,"[{u'position': u'Contestant.', u'organization'...","[{u'prev_field': u'Mathematik', u'prev_school'..."


In [105]:
from pandas.io.json import json_normalize  

In [19]:
friends_data = []
for i in range(1,8):
    with open('friends_data/friends_data_page' + str(i) + '.json', 'r') as f:
        friends_data += json.load(f)

In [20]:
with open('friends_data/friends_data_all.json', 'w') as f:
    json.dump(friends_data, f, ensure_ascii=False)